<a href="https://colab.research.google.com/github/rahiakela/natural-language-processing-research-and-practice/blob/main/real-world-natural-language-processing/2-first-nlp-application/sst_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Introducing sentiment analysis

In a scenario, you wanted to extract users’ subjective opinions
from online survey results. You have a collection of textual data in response to a
free-response question, but you are missing the answers to the “How do you like our
product?” question, which you’d like to recover from the text. 

This task is called sentiment
analysis, which is a text analytic technique used in the automatic identification
and categorization of subjective information within text. The technique is widely used
in quantifying opinions, emotions, and so on that are written in an unstructured way
and, thus, hard to quantify otherwise. Sentiment analysis is applied to a wide variety of
textual resources such as survey, reviews, and social media posts.

In machine learning, classification means categorizing something into a set of predefined,
discrete categories. One of the most basic tasks in sentiment analysis is the
classification of polarity, that is, to classify whether the expressed opinion is positive,
negative, or neutral.

<img src='images/1.png?raw=1' width='800'/>



##Setup

In [ ]:
!pip -q install allennlp==2.5.0
!pip -q install allennlp-models==2.5.0
!git -q clone https://github.com/mhagiwara/realworldnlp.git
%cd realworldnlp

In [2]:
from itertools import chain
from typing import Dict

import numpy as np
import torch
import torch.optim as optim
from allennlp.data.data_loaders import MultiProcessDataLoader
from allennlp.data.samplers import BucketBatchSampler
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.nn.util import get_text_field_mask
from allennlp.training import GradientDescentTrainer
from allennlp.training.metrics import CategoricalAccuracy, F1Measure
from allennlp_models.classification.dataset_readers.stanford_sentiment_tree_bank import \
    StanfordSentimentTreeBankDatasetReader

from realworldnlp.predictors import SentenceClassifierPredictor

In [3]:
EMBEDDING_DIM = 128
HIDDEN_DIM = 128

##What is a dataset?

In NLP, records in a dataset are usually some type of linguistic units, such as words,
sentences, or documents. A dataset of natural language texts is called a corpus (plural: corpora).

If a dataset contains a collection of sentences annotated
with their parse trees, the dataset is called a treebank. The most famous example
of this is [Penn Treebank (PTB)](http://realworldnlpbook.com/ch2.html#ptb), which
has been serving as the de facto standard dataset for training and evaluating NLP tasks
such as part-of-speech (POS) tagging and parsing.

A closely related term to a record is an instance. In machine learning, an instance is
a basic unit for which the prediction is made.

<img src='images/2.png?raw=1' width='800'/>

Finally, a label is a piece of information
attached to some linguistic unit in a dataset.

Labels are usually used as training signals (i.e., answers for
the training algorithm) in a supervised machine learning setting.



###Train, validation, and test sets

A train (or training) set is the main dataset used to train the NLP/ML models.
Instances from the train set are usually fed to the ML training pipeline directly and
used to learn parameters of the model.

A validation set (also called a dev or development set) is used for model selection. Model
selection is a process where appropriate NLP/ML models are selected among all possible
models that can be trained using the train set, and here’s why it’s necessary.

<img src='images/3.png?raw=1' width='800'/>

In summary, when training NLP models, use a train set to train your model candidates,
use a validation set to choose good ones, and use a test set to evaluate them.

###Loading SST datasets using AllenNLP

AllenNLP already supports an abstraction called DatasetReader, which takes care of
reading a dataset from the original format (be it raw text or some exotic XML-based
format) and returns it as a collection of instances. 

We are going to use Stanford-
SentimentTreeBankDatasetReader(), which is a type of DatasetReader that
specifically deals with SST datasets, as shown here:

In [4]:
reader = StanfordSentimentTreeBankDatasetReader()
train_path = 'https://s3.amazonaws.com/realworldnlpbook/data/stanfordSentimentTreebank/trees/train.txt'
dev_path = 'https://s3.amazonaws.com/realworldnlpbook/data/stanfordSentimentTreebank/trees/dev.txt'

##Using word embeddings

In [ ]:
# Model in AllenNLP represents a model that is trained.
class LstmClassifier(Model):
    def __init__(self,
                 embedder: TextFieldEmbedder,
                 encoder: Seq2VecEncoder,
                 vocab: Vocabulary,
                 positive_label: str = '4') -> None:
        super().__init__(vocab)
        # We need the embeddings to convert word IDs to their vector representations
        self.embedder = embedder

        self.encoder = encoder

        # After converting a sequence of vectors to a single vector, we feed it into
        # a fully-connected linear layer to reduce the dimension to the total number of labels.
        self.linear = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('labels'))

        # Monitor the metrics - we use accuracy, as well as prec, rec, f1 for 4 (very positive)
        positive_index = vocab.get_token_index(positive_label, namespace='labels')
        self.accuracy = CategoricalAccuracy()
        self.f1_measure = F1Measure(positive_index)

        # We use the cross entropy loss because this is a classification task.
        # Note that PyTorch's CrossEntropyLoss combines softmax and log likelihood loss,
        # which makes it unnecessary to add a separate softmax layer.
        self.loss_function = torch.nn.CrossEntropyLoss()

    # Instances are fed to forward after batching.
    # Fields are passed through arguments with the same name.
    def forward(self,
                tokens: Dict[str, torch.Tensor],
                label: torch.Tensor = None) -> torch.Tensor:
        # In deep NLP, when sequences of tensors in different lengths are batched together,
        # shorter sequences get padded with zeros to make them equal length.
        # Masking is the process to ignore extra zeros added by padding
        mask = get_text_field_mask(tokens)

        # Forward pass
        embeddings = self.embedder(tokens)
        encoder_out = self.encoder(embeddings, mask)
        logits = self.linear(encoder_out)

        # In AllenNLP, the output of forward() is a dictionary.
        # Your output dictionary must contain a "loss" key for your model to be trained.
        output = {"logits": logits}
        if label is not None:
            self.accuracy(logits, label)
            self.f1_measure(logits, label)
            output["loss"] = self.loss_function(logits, label)

        return output

    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        return {'accuracy': self.accuracy.get_metric(reset),
                **self.f1_measure.get_metric(reset)}

In [ ]:
reader = StanfordSentimentTreeBankDatasetReader()
train_path = 'https://s3.amazonaws.com/realworldnlpbook/data/stanfordSentimentTreebank/trees/train.txt'
dev_path = 'https://s3.amazonaws.com/realworldnlpbook/data/stanfordSentimentTreebank/trees/dev.txt'

In [ ]:
sampler = BucketBatchSampler(batch_size=32, sorting_keys=["tokens"])
train_data_loader = MultiProcessDataLoader(reader, train_path, batch_sampler=sampler)
dev_data_loader = MultiProcessDataLoader(reader, dev_path, batch_sampler=sampler)

In [ ]:
# You can optionally specify the minimum count of tokens/labels.
# `min_count={'tokens':3}` here means that any tokens that appear less than three times
# will be ignored and not included in the vocabulary.
vocab = Vocabulary.from_instances(chain(train_data_loader.iter_instances(), dev_data_loader.iter_instances()),
                                  min_count={'tokens': 3})

In [ ]:
train_data_loader.index_with(vocab)
dev_data_loader.index_with(vocab)

In [ ]:
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)

In [ ]:
# BasicTextFieldEmbedder takes a dict - we need an embedding just for tokens,
# not for labels, which are used as-is as the "answer" of the sentence classification
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})

In [ ]:
# Seq2VecEncoder is a neural network abstraction that takes a sequence of something
# (usually a sequence of embedded word vectors), processes it, and returns a single
# vector. Oftentimes this is an RNN-based architecture (e.g., LSTM or GRU), but
# AllenNLP also supports CNNs and other simple architectures (for example,
# just averaging over the input vectors).
encoder = PytorchSeq2VecWrapper(
    torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))

In [ ]:
model = LstmClassifier(word_embeddings, encoder, vocab)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [ ]:
trainer = GradientDescentTrainer(
    model=model,
    optimizer=optimizer,
    data_loader=train_data_loader,
    validation_data_loader=dev_data_loader,
    patience=10,
    num_epochs=20,
    cuda_device=-1)

trainer.train()

In [ ]:
predictor = SentenceClassifierPredictor(model, dataset_reader=reader)
logits = predictor.predict('This is the best movie ever!')['logits']
label_id = np.argmax(logits)

print(model.vocab.get_token_from_index(label_id, 'labels'))